### Gloden cross strategy

In [1]:
import time
from datetime import datetime
import vectorbt as vbt

import pandas as pd
import pandas_ta as ta
import numpy as np
import mplfinance as mpf

### Import csv

In [17]:
data = pd.read_csv('AUDUSD_15m.csv')
data.tail()

,timestamp,open,high,low,close,adj close,volumn
11930,2022-04-29 16:45:00+01:00,0.710424,0.711288,0.710252,0.710308,0.710308,0
11931,2022-04-29 17:00:00+01:00,0.710252,0.710252,0.709084,0.709210,0.709210,0
11932,2022-04-29 17:15:00+01:00,0.709144,0.709824,0.709144,0.709310,0.709310,0
11933,2022-04-29 17:30:00+01:00,0.709215,0.709320,0.708843,0.709044,0.709044,0
11934,2022-04-29 17:45:00+01:00,0.709134,0.709471,0.708205,0.708386,0.708386,0


In [18]:
df = data.copy()

### Create strategy

In [19]:
percent_r = ta.Strategy(
    name = 'Willams % R',
    description = 'Willams % R Strategy',
    ta = [
        {'kind': 'ema', 'length': 100},
        {'kind': 'willr', 'length': 20},
        {'kind': 'atr', 'length': 14}
    ]
)
df.ta.strategy(percent_r)

In [20]:
df

,timestamp,open,high,low,close,adj close,volumn,EMA_100,WILLR_20,ATRr_14
0,2021-11-04 17:00:00+00:00,0.739776,0.739973,0.739536,0.739919,0.739919,0,NaN,NaN,NaN
1,2021-11-04 17:15:00+00:00,0.739793,0.740192,0.739661,0.739776,0.739776,0,NaN,NaN,NaN
2,2021-11-04 17:30:00+00:00,0.739826,0.739973,0.739486,0.739919,0.739919,0,NaN,NaN,NaN
3,2021-11-04 17:45:00+00:00,0.739722,0.739973,0.739322,0.739656,0.739656,0,NaN,NaN,NaN
4,2021-11-04 18:00:00+00:00,0.739738,0.739919,0.739497,0.739864,0.739864,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
11930,2022-04-29 16:45:00+01:00,0.710424,0.711288,0.710252,0.710308,0.710308,0,0.713454,-89.615701,0.001238
11931,2022-04-29 17:00:00+01:00,0.710252,0.710252,0.709084,0.709210,0.709210,0,0.713370,-98.604430,0.001237
11932,2022-04-29 17:15:00+01:00,0.709144,0.709824,0.709144,0.709310,0.709310,0,0.713289,-97.487444,0.001197
11933,2022-04-29 17:30:00+01:00,0.709215,0.709320,0.708843,0.709044,0.709044,0,0.713205,-97.653728,0.001146


### Create signal¶

In [21]:
class BacktestWilliamsR:
    def __init__(self, params):
        self.orders = pd.DataFrame(columns = ['action', 'open time', 'open', 'close time', 'close', 'T/P', 'S/L', 'result'])
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        self.unknow_result = []
        self.willr_status = 'idle' # idle, overbought, oversold
        
        self.current_action = 'close'
        self.atr = params['atr']
        self.ema = params['ema']
        self.willr = params['willr']
        
        self.overbought = params['overbought']
        self.oversold = params['oversold']
        
        self.pip_value = params['pip_value']
        self.rr = params['rr']
        self.df = params['df']
        self.plot = params['plot']
        
    def reset_order(self):
        self.order = {'action': None, 'open time': None, 'open': None, 'close time': None, \
                      'close': None, 'T/P': None, 'S/L': None,'result': None}
        
    def get_df(self):
        return self.df
        
    def plot_order(self, order):
        pass
        
    def backtest(self):
        n_bars = 3
        percent_r = ta.Strategy(
            name = 'Willams % R',
            description = 'Willams % R Strategy',
            ta = [
                {'kind': 'ema', 'length': self.ema},
                {'kind': 'willr', 'length': self.willr},
                {'kind': 'atr', 'length': self.atr}
            ]
        )
        self.df.ta.strategy(percent_r)
        self.df = self.df.copy().dropna()
        self.df['timestamp'] = pd.to_datetime(self.df['timestamp'])
        self.df['action'] = None
        
        self.df['entries'] = False
        self.df['exits'] = False
        self.df['short_entries'] = False
        self.df['short_exits'] = False
        self.df['price_action'] = self.df['close']
        
        for index, row in self.df.iterrows():
            entry = 0
            
            if self.current_action == 'close':
                
                if self.willr_status == 'idle':
                    if (row['EMA_'+str(self.ema)] < row['close']) and (row['WILLR_'+str(self.willr)] < self.oversold):
                        self.willr_status = 'oversold'
                        
                    elif (row['EMA_'+str(self.ema)] > row['close']) and (row['WILLR_'+str(self.willr)] > self.overbought):
                        self.willr_status = 'overbought'
                        
                elif self.willr_status == 'oversold':
                    if (row['EMA_'+str(self.ema)] < row['close']) and (row['WILLR_'+str(self.willr)] > -50):
                        self.df.at[index, 'action'] = 'buy'
                        self.df.at[index, 'entries'] = True
                        self.df.at[index, 'price_action'] = row['open']
                        self.order['action'] = 'buy'
                                
                        self.order['open time'] = row['timestamp']
                        self.order['open'] = row['close']
                        self.order['S/L'] = row['open'] - row['ATRr_'+str(self.atr)]
                        self.order['T/P'] = row['open'] + (self.rr * abs(row['open'] - self.order['S/L']))
                        self.current_action = 'buy'
                        
                        self.willr_status = 'idle'
                        
                    elif (row['EMA_'+str(self.ema)] > row['close']):
                        self.willr_status = 'idle'
                        
                elif self.willr_status == 'overbought':
                    if (row['EMA_'+str(self.ema)] > row['close']) and (row['WILLR_'+str(self.willr)] < -50):
                        self.df.at[index, 'action'] = 'sell'
                        self.df.at[index, 'short_entries'] = True
                        self.df.at[index, 'price_action'] = row['open']
                        self.order['action'] = 'sell'
                                
                        self.order['open time'] = row['timestamp']
                        self.order['open'] = row['close']
                        self.order['S/L'] = row['open'] + row['ATRr_'+str(self.atr)]
                        self.order['T/P'] = row['open'] - (self.rr * abs(row['open'] - self.order['S/L']))
                        self.current_action = 'sell'
                        
                        self.willr_status = 'idle'
                        
                    elif (row['EMA_'+str(self.ema)] < row['close']):
                        self.willr_status = 'idle'
        
            if self.current_action == 'buy':
                if row['low'] <= self.order['S/L'] and row['high'] >= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['low'] <= self.order['S/L']:
                    self.df.at[index, 'exits'] = True
                    self.df.at[index, 'price_action'] = row['low']
                    
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()
                    
                elif row['high'] >= self.order['T/P']:
                    self.df.at[index, 'exits'] = True
                    self.df.at[index, 'price_action'] = row['high']
                    
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()

            elif self.current_action == 'sell':
                if row['high'] >= self.order['S/L'] and row['low'] <= self.order['T/P']:
                    self.unknow_result.append(row)
                    
                if row['high'] >= self.order['S/L']:
                    self.df.at[index, 'short_exits'] = True
                    self.df.at[index, 'price_action'] = row['high']
                    
                    self.order['result'] = 'S/L'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['S/L']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()
                    
                elif row['low'] <= self.order['T/P']:
                    self.df.at[index, 'short_exits'] = True
                    self.df.at[index, 'price_action'] = row['low']
                    
                    self.order['result'] = 'T/P'
                    self.order['close time'] = row['timestamp']
                    self.order['close'] = self.order['T/P']
                    
                    self.current_action = 'close'
                    self.orders = self.orders.append(self.order, ignore_index = True)
                    if self.plot == True: self.plot_order(self.order)
                    self.reset_order()

        try:
            win_rate = round(len(self.orders[self.orders['result'] == 'T/P']) * 100 / len(self.orders), 2)
            loss_rate = round(len(self.orders[self.orders['result'] == 'S/L']) * 100 / len(self.orders), 2)

            win_orders = int((win_rate * len(self.orders)) / 100 )
            loss_orders = len(self.orders) - win_orders
            gain = (win_orders * self.rr) - loss_orders

            result_info = f"total orders: {len(self.orders)}\nema: {self.ema}, willr: {self.willr}, atr: {self.atr}, overbought: {self.overbought}, oversold: {self.oversold}, gain: {gain}\n"
            return {'win_rate': win_rate, 'info': result_info, 'total_order': len(self.orders), 'rr': self.rr, 'gain': gain, 'unknow': self.unknow_result}
        except:
    #         print("No orders to action")
            return {'win_rate': 0, 'info': 'No orders to action'}

In [22]:
pair_test = BacktestWilliamsR({
    'atr': 14,
    'ema': 100,
    'willr': 20,
    'overbought': -20,
    'oversold': -80,
    'pip_value': 0.001,
    'rr': 1.5,
    'plot': False,
    'df': df.copy()
})
pair_test.backtest()

{'win_rate': 75.78,
 'info': 'total orders: 128\nema: 100, willr: 20, atr: 14, overbought: -20, oversold: -80, gain: 112.0\n',
 'total_order': 128,
 'rr': 1.5,
 'gain': 112.0,
 'unknow': []}

In [23]:
"""
Grid parameter example
willr: 10 -> 40
overbought oversold: 70->90, 20->30
"""
def create_grid_params(line1_start, line1_end, line2_start, line2_end):
    params = []
    for i in range(line1_start, line1_end+1):
        for j in range(line2_start, line2_end+1):
            params.append([i, j])
    return params
params_list = create_grid_params(10, 40, 10, 30)
len(params_list)

651

In [24]:
good_params = {'win': 0, 'info': ''}
start = 0
end = 651
win_start = 0

loop_num = start
for param in params_list[start:end]:
    obj = BacktestWilliamsR({'atr': 14, 'ema': 100,  'willr': param[0],'overbought': -1*param[1], 'oversold': -1*(100-param[1]), 'pip_value': 0.001, 'rr': 1.5, 'df': df.copy(), 'plot': False})
    result = obj.backtest()
    if result['win_rate'] > win_start and result['total_order'] > 300:
        win_start = result['win_rate']
        good_params['win'] = result['win_rate']
        good_params['info'] = result['info']
    loop_num = loop_num + 1
    print(f"run: {loop_num} to {end}", end = "\r")
    
print(f"\nwin rate: {good_params['win']} %\n{good_params['info']}")

run: 651 to 651
win rate: 68.2 %
total orders: 305
ema: 100, willr: 12, atr: 14, overbought: -29, oversold: -71, gain: 215.0



### ETHUSDT 15m
* win rate: 79.69 %
* total orders: 256
* ema: 100, willr: 37, atr: 14, overbought: -14, oversold: -86, gain: 254.0

### GBPUSD 15m
* win rate: 73.28 %
* total orders: 131
* ema: 100, willr: 17, atr: 14, overbought: -15, oversold: -85, gain: 106.5

* win rate: 64.33 %
* total orders: 342
* ema: 100, willr: 10, atr: 14, overbought: -29, oversold: -71, gain: 208.0

### USDJPY 15m

* win rate: 65.38 %
* total orders: 130
* ema: 100, willr: 12, atr: 14, overbought: -17, oversold: -83, gain: 80.0

### AUDUSD 15m
* win rate: 76.92 %
* total orders: 130
* ema: 100, willr: 29, atr: 14, overbought: -30, oversold: -70, gain: 117.5

### EURUSD 15m
* win rate: 69.29 %
* total orders: 127
* ema: 100, willr: 24, atr: 14, overbought: -25, oversold: -75, gain: 90.5

* win rate: 61.84 %
* total orders: 304
* ema: 100, willr: 12, atr: 14, overbought: -30, oversold: -70, gain: 163.5